<a href="https://colab.research.google.com/github/djdev/workshop1_data_ingestion/blob/main/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [40]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

# What is the sum of the outputs of the generator for limit = 5?
print('\nQuestion 1: What is the sum of the outputs of the generator for limit = 5?')
print(__builtins__.sum(square_root_generator(limit)))


# What is the 13th number yielded
limit = 13
generator_2 = square_root_generator(limit)

sqrts = []
for sqrt_value in generator_2:
    sqrts.append(sqrt_value)

print('\nQuestion 2: What is the 13th number yielded')
print(sqrts[limit - 1]) # the 13th value



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979

Question 1: What is the sum of the outputs of the generator for limit = 5?
8.382332347441762

Question 2: What is the 13th number yielded
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [41]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [42]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [71]:
# to do: homework :)
import os
import dlt
import duckdb

# Set the bucket_url. We can also use a local folder
os.environ['DESTINATION__FILESYSTEM__BUCKET_URL'] = 'file:///content/.dlt/my_folder'

pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# Question 3. Append the 2 generators
info = pipeline.run(people_1, table_name="appended_users", write_disposition="replace")
info = pipeline.run(people_2, table_name="appended_users", write_disposition="append")
print(info)

# Question 4. Merge the 2 generators using the ID column.
info_by_id = pipeline.run(people_1, table_name="merged_users", write_disposition="replace")
info_by_id = pipeline.run(people_2, table_name="merged_users", write_disposition="merge", primary_key="ID")
print(info_by_id)

# show the outcome
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n appended_users table below:")
appended_users_table = conn.sql("SELECT * FROM appended_users").df()
display(appended_users_table)

# Question 3. calculate the sum of all ages of people.
total_age_appended_users = conn.sql("SELECT SUM(age) FROM appended_users").df()
print(total_age_appended_users)


print("\n\n\n merged_users table")
merged_users_table = conn.sql("SELECT * FROM merged_users").df()
display(merged_users_table)

# Question 4. calculate the sum of all ages of people.
total_age_merged_users = conn.sql("SELECT SUM(age) FROM merged_users").df()
print(total_age_merged_users)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708303929.6280189 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708303931.1008062 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ append_users        │
│ appended_users      │
│ merged_users        │
│ people1             │
│ people_id           │
└─────────────────────┘




 appended_users table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708303928.7128956,Kaw7yqtKD9MzIg,None
1,2,Person_2,27,City_A,1708303928.7128956,A1mP5+WQ0B/kBA,None
2,3,Person_3,28,City_A,1708303928.7128956,1b84LitneUvBQw,None
3,4,Person_4,29,City_A,1708303928.7128956,f67LwyZBJjFWMA,None
4,5,Person_5,30,City_A,1708303928.7128956,c5hdY/HAXQSJng,None
5,3,Person_3,33,City_B,1708303929.6280189,fu7ofm291uB/JQ,Job_3
6,4,Person_4,34,City_B,1708303929.6280189,tc2gzgh08VnC+w,Job_4
7,5,Person_5,35,City_B,1708303929.6280189,leaCVaSgYe7GZQ,Job_5
8,6,Person_6,36,City_B,1708303929.6280189,N8l0L+8ic9UNhw,Job_6
9,7,Person_7,37,City_B,1708303929.6280189,dqhGW/BJzTL3Yw,Job_7


   sum(age)
0     353.0



 merged_users table


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708303930.3858302,HH2BQfAjfeKXFg,None
1,2,Person_2,27,City_A,1708303930.3858302,4dM5myVm8JRdZQ,None
2,8,Person_8,38,City_B,1708303931.1008062,E/MB8+mLvfkYrw,Job_8
3,7,Person_7,37,City_B,1708303931.1008062,JSWTSG6mpuCAGw,Job_7
4,3,Person_3,33,City_B,1708303931.1008062,8fusq6gShEAgdQ,Job_3
5,6,Person_6,36,City_B,1708303931.1008062,Y3bjppR/9/S2hQ,Job_6
6,5,Person_5,35,City_B,1708303931.1008062,+Lz6V7yyYs+6uw,Job_5
7,4,Person_4,34,City_B,1708303931.1008062,ncIwmhQ3ZiydVQ,Job_4


   sum(age)
0     266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX